In [ ]:
import networkx
import random
import scipy

def generate_graph(n, seed=1):
    graph = networkx.Graph()
    edge_list = []
    for n1 in range(n):
        for n2 in range(n1+1,n):
            edge_list.append((n1,n2))
    
    graph.add_edges_from(edge_list)
    random.seed(seed)
    weights = [random.random() for i in range(len(edge_list))]

    for index, edge in enumerate(graph.edges()):
        graph.get_edge_data(*edge)['weight'] = weights[index]

    return graph, weights

no_vertices = 5
graph = generate_graph(no_vertices)[0]
pos=networkx.circular_layout(graph)
networkx.draw_networkx(graph, pos)
labels = networkx.get_edge_attributes(graph,'weight')
for edge in labels:
    labels[edge] = round(labels[edge], 3)
tmp = networkx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)


In [ ]:
from src_code import build_operators

hamiltonian = build_operators.cut_hamiltonian(graph=graph)
gradient_ops_dict = build_operators.build_all_mixers(graph=graph)
pauli_ops_dict = build_operators.build_all_paulis(no_vertices)

In [ ]:
from src_code import useful_methods

max_cut_solution = useful_methods.find_optimal_cut(graph)
max_cut_value = max_cut_solution[1]
max_ham_eigenvalue = max_cut_solution[2]
ham_offset = max_cut_value - max_ham_eigenvalue

hamiltonian = build_operators.cut_hamiltonian(graph)
mix_hamiltonian = build_operators.mix_hamiltonian(graph)
det_ham = scipy.linalg.norm(hamiltonian.toarray(),2)
det_mix = scipy.linalg.norm(mix_hamiltonian.toarray(),2)

beta = 0
delta_t = 0.1
ham_approx_ratios = []
cut_approx_ratios = []

max_layers = 100
layer = 0

curr_dens_mat = build_operators.initial_density_matrix(no_vertices)

In [ ]:
def build_layer(curr_dens_mat,beta, delta_t):
    """ one layer 
    """
    cut_unit = build_operators.cut_unitary(graph,delta_t,pauli_ops_dict)
    curr_dens_mat = (cut_unit * curr_dens_mat) * (cut_unit.transpose().conj())

    mix_unit = build_operators.mixer_unitary('standard_x', beta * delta_t, pauli_ops_dict, no_vertices)
    curr_dens_mat = (mix_unit * curr_dens_mat) * (mix_unit.transpose().conj())

    return curr_dens_mat

In [ ]:
def update_time(curr_dens_mat):

    A = 1j*(mix_hamiltonian*hamiltonian - hamiltonian * mix_hamiltonian) * curr_dens_mat
    expectation = A.trace().real
    delta_t = abs(expectation) / (2*(2* det_ham * det_mix + abs(expectation)) * (det_ham + det_mix * beta))
    print(delta_t, end=' ')
    return delta_t 

In [ ]:
while layer < max_layers:
    layer += 1
    curr_dens_mat = build_layer(curr_dens_mat, beta, delta_t)
    if beta == 0:
        beta = 10
    delta_t = update_time(curr_dens_mat)

    hamiltonian_expectation = (hamiltonian * curr_dens_mat).trace().real
    ham_approx_ratio = hamiltonian_expectation / max_ham_eigenvalue
    cut_approx_ratio = (hamiltonian_expectation + max_cut_value - max_ham_eigenvalue) / max_cut_value

    print("layer", layer, ": ", cut_approx_ratio,sep='')